In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input,Conv2D, MaxPool2D, Flatten, Dropout
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2 as cv

2025-01-21 09:49:51.215494: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 09:49:51.217131: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 09:49:51.249280: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 09:49:51.282880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737433191.339253   23649 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737433191.36

In [2]:
NUM_CLASSES = 16
MAX_EPOCHS = 10
PICKLE_FILE = 'dataset/dataset_large.pickle'
MAP_SYMBOLS = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
               '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
               '+': 10, '-': 11, 
              'times': 12, 'div': 13, '(': 14, ')': 15}
VEC_SYMBOLS = np.vectorize(MAP_SYMBOLS.get)

## data load, split into train and testing sets and reshaping

In [3]:
with open(PICKLE_FILE, 'rb') as f:
    data = pickle.load(f)

X = np.array(data['img'])
y = VEC_SYMBOLS(np.array(data['label']))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

## CNN model

In [4]:
model = Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(Conv2D(30, (5, 5), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(15, (3, 3), activation ='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation ='relu'))
model.add(Dense(50, activation ='relu'))
model.add(Dense(NUM_CLASSES, activation ='softmax'))
model.compile(loss ='categorical_crossentropy', 
              optimizer ='adam', metrics =['accuracy'])

2025-01-21 09:51:52.598315: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [7]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, min_delta=0.01)
mc = ModelCheckpoint('rmodel.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
cb = [es, mc]

In [8]:
history = model.fit(X_train, y_train, epochs=MAX_EPOCHS, callbacks=cb, batch_size=200, shuffle=True, verbose=1, validation_split=0.1)

Epoch 1/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8581 - loss: 0.5828
Epoch 1: val_accuracy improved from -inf to 0.98788, saving model to rmodel.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 25s 64ms/step - accuracy: 0.8587 - loss: 0.5806 - val_accuracy: 0.9879 - val_loss: 0.0526
Epoch 2/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9872 - loss: 0.0493
Epoch 2: val_accuracy improved from 0.98788 to 0.99237, saving model to rmodel.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.9872 - loss: 0.0493 - val_accuracy: 0.9924 - val_loss: 0.0335
Epoch 3/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9917 - loss: 0.0323
Epoch 3: val_accuracy improved from 0.99237 to 0.99442, saving model to rmodel.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.9917 - loss: 0.0323 - val_accuracy: 0.9944 - val_loss: 0.0254
Epoch 4/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9941 - loss: 0.0245
Epoch 4: val_accuracy improved from 0.99442 to 0.99496, saving model to rmodel.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 22s 66ms/step - accuracy: 0.9941 - loss: 0.0245 - val_accuracy: 0.9950 - val_loss: 0.0260
Epoch 5/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9948 - loss: 0.0194
Epoch 5: val_accuracy did not improve from 0.99496
331/331 ━━━━━━━━━━━━━━━━━━━━ 22s 65ms/step - accuracy: 0.9948 - loss: 0.0194 - val_accuracy: 0.9940 - val_loss: 0.0254
Epoch 6/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9948 - loss: 0.0194
Epoch 6: val_accuracy did not improve from 0.99496
331/331 ━━━━━━━━━━━━━━━━━━━━ 21s 64ms/step - accuracy: 0.9948 - loss: 0.0194 - val_accuracy: 0.9950 - val_loss: 0.0281
Epoch 7/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9964 - loss: 0.0141
Epoch 7: val_accuracy did not improve from 0.99496
331/331 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9964 - loss: 0.0141 - val_accuracy: 0.9948 - val_loss: 0.0216
Epoch 8/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9961 - loss: 0.0153
Epoch 8: val_accuracy i

331/331 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.9961 - loss: 0.0153 - val_accuracy: 0.9955 - val_loss: 0.0194
Epoch 9/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9969 - loss: 0.0134
Epoch 9: val_accuracy improved from 0.99551 to 0.99632, saving model to rmodel.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 22s 66ms/step - accuracy: 0.9969 - loss: 0.0134 - val_accuracy: 0.9963 - val_loss: 0.0191
Epoch 10/10
330/331 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.9969 - loss: 0.0139
Epoch 10: val_accuracy improved from 0.99632 to 0.99687, saving model to rmodel.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.9969 - loss: 0.0139 - val_accuracy: 0.9969 - val_loss: 0.0190


In [9]:
model.evaluate(X_test, y_test)

574/574 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9972 - loss: 0.0113


[0.012227257713675499, 0.9971675872802734]

In [10]:
model.save('rmodel.h5')